In [1]:
import arcpy
import subprocess
# Module to download files with an URL
import urllib.request
# Zip utlity to extract files
from zipfile import ZipFile
# pandas to view table and possibly analyze data in the future?
import pandas as pd

############### Project properties ###############

# Root GIS directory
# Got to use double backslash because of other modules
root = "c:\\MontgomeryGIS"

# Project name - creates a folder in project directory
project = "SkyBridge2"

# Name of area of interest as a single point and buffer distance from point
# aoi = f"{root}\\data\\locations.gdb\\courthouse"
# OR 
long = -83.578886 
lat = 37.818864
buffer_distance = 1000

# NAIP edition
naip_year = "2018" # 2018, 2016, 2006 (all 2-ft resolution orthophoto)

############### Local existing assets ###############

# Locations of index grids
las_grid = f"{root}\\data\\GDB\\Kentucky_5k_PointCloudGrid.shp"
naip_grid = f"{root}\\data\\GDB\\Kentucky_10K_NAIP.shp"

# Location of the laszip64.exe utility
las_tools = f"{root}\\laszip64.exe"

# Location of the PotreeConverter.exe utility
potree_tools = f"{root}\\data\\Potree\\PotreeConverter\\PotreeConverter.exe"

############### Project assets that will be created ###############

# Output geodatabase name
geodb = "SkyBridge2.gdb"

# Downloads folder
downloads = f'{root}\\{project}\\downloads\\'
lidar = f'{root}\\{project}\\lidar\\'
lidar_extract = f'{root}\\{project}\\lidar_extract\\'
lidar_color = f'{root}\\{project}\\lidar_color\\'

# LAS dataset name, temp list, and class codes
las_dataset = f'{lidar}\\{project}.lasd'

arcpy.env.overwriteOutput = True
spatial_reference = arcpy.Describe(las_grid).spatialReference

In [2]:
# Create folders
folders = [f'{root}\\{project}', downloads, lidar, lidar_extract, lidar_color]
for folder in folders:
    subprocess.run(f'mkdir {folder}', shell=True, stdout=subprocess.PIPE)
    print(f"mkdir {folder}")

# Create project geodatabase
arcpy.CreateFileGDB_management(f'{root}\\{project}', geodb)

# Create project default geodatabase
arcpy.env.workspace = f'{root}\\{project}\\{geodb}'

# Show contents of project directory
completed = subprocess.run(f'dir {root}\\{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

# Using area of interest layer or lat/long?
try:
    aoi
    print(f"Using {aoi}")
except:
    print(f"Creating point at lat: {lat}, long: {long}")
    
    # Create a feature class with a spatial reference of GCS WGS 1984
    result = arcpy.management.CreateFeatureclass(
        arcpy.env.workspace, 
        "test", "POINT", spatial_reference=4326)
    feature_class = result[0]
    
    # Insert lat/long point
    with arcpy.da.InsertCursor(feature_class, ['SHAPE@']) as cursor:
        cursor.insertRow([(long, lat)])
    
    # Create area of interest and remove temp layer
    aoi = "aoi"
    arcpy.Project_management("test", aoi, spatial_reference)
    arcpy.Delete_management ("test")

# Buffer aoi
arcpy.Buffer_analysis(aoi, f'{project}_{buffer_distance}ft', buffer_distance)

# Create a temp layer to find which LAS files to download
arcpy.Intersect_analysis ([f'{project}_{buffer_distance}ft', las_grid], "temp")
layers = arcpy.ListFeatureClasses()
for layer in layers:
    print(f"created {layer}")

# Find URLs and download them and use laszip64.exe to convert 
cursor = arcpy.da.SearchCursor("temp", ['ftppath', 'LASVersion', 'Year'])
i = 0
las_names = []
for row in cursor:
    url = row[0]
    name = url[-12:]
    las_names.append(f'{lidar}{url[-12:-4]}.las')
    with urllib.request.urlopen(url) as response: 
        print(f'Downloading {url}')
        with open(f'{downloads}{name}', 'wb') as outFile:
            data = response.read()
            outFile.write(data)   
    print(f'Using: {las_tools} -i {downloads}{name} -o {las_names[i]}')
    completed = subprocess.run(f'{las_tools} -i {downloads}{name} -o {las_names[i]}', shell=True, stdout=subprocess.PIPE)
    i += 1

arcpy.CreateLasDataset_management (las_names, las_dataset, "#", "#", spatial_reference, True, True)  

# Find code statistics to decide which points include in derivatives
arcpy.LasDatasetStatistics_management (las_dataset, "#", f'{root}\\{project}\\stats.csv', "#", "#", "#")
with open(f'{root}\\{project}\\stats.csv', encoding='utf-8') as csv:
    reader = pd.read_csv(csv)
    pdData = pd.DataFrame(reader)

pdData[pdData["Category"] == "ClassCodes"]

mkdir c:\MontgomeryGIS\SkyBridge2
mkdir c:\MontgomeryGIS\SkyBridge2\downloads\
mkdir c:\MontgomeryGIS\SkyBridge2\lidar\
mkdir c:\MontgomeryGIS\SkyBridge2\lidar_extract\
mkdir c:\MontgomeryGIS\SkyBridge2\lidar_color\
 Volume in drive C has no label.
 Volume Serial Number is 6020-6EFD

 Directory of c:\MontgomeryGIS\SkyBridge2

11/24/2019  02:11 PM    <DIR>          .
11/24/2019  02:11 PM    <DIR>          ..
11/24/2019  02:11 PM    <DIR>          downloads
11/24/2019  02:11 PM    <DIR>          lidar
11/24/2019  02:11 PM    <DIR>          lidar_color
11/24/2019  02:11 PM    <DIR>          lidar_extract
11/24/2019  02:11 PM    <DIR>          SkyBridge2.gdb
               0 File(s)              0 bytes
               7 Dir(s)  358,467,907,584 bytes free

Creating point at lat: 37.818864, long: -83.578886
created aoi
created SkyBridge2_1000ft
created temp
Using: c:\MontgomeryGIS\laszip64.exe -i c:\MontgomeryGIS\SkyBridge2\downloads\N106E354.laz -o c:\MontgomeryGIS\SkyBridge2\lidar\N106E354

,Item,Category,Pt_Cnt,Percent,Z_Min,Z_Max,Intensity_Min,Intensity_Max,Synthetic_Pt_Cnt,Range_Min,Range_Max
12,1_Unclassified,ClassCodes,35453840.0,87.09,702.77,1340.90,0.0,255.0,0.0,NaN,NaN
13,2_Ground,ClassCodes,5244394.0,12.88,702.26,1256.59,0.0,255.0,0.0,NaN,NaN
14,7_Low_Point(noise),ClassCodes,11477.0,0.03,702.32,2241.30,0.0,242.0,0.0,NaN,NaN


In [3]:
las_ground = [2]
las_trees = [1, 2]

# Filter for ground points 
arcpy.MakeLasDatasetLayer_management (las_dataset, f'{lidar}ground', las_ground)
# Filter for ground and above ground points
arcpy.MakeLasDatasetLayer_management (las_dataset, f'{lidar}trees', las_trees)

# Ceate DEM and hillshade
arcpy.LasDatasetToRaster_conversion (f'{lidar}ground', f'{project}_dem_5ft', "#", "#", "#", "#", 5)
arcpy.HillShade_3d(f'{project}_dem_5ft', f'{project}_hillshade', 270, 55)

# Create a temp layer to find which NAIP files to download
arcpy.RasterDomain_3d (f'{project}_hillshade', 'domain', 'POLYGON')
arcpy.Intersect_analysis (['domain', naip_grid], "temp")

# Find orthophoto URLs, download, and extract 
cursor = arcpy.da.SearchCursor("temp", ['ftppath16', 'TileName'])
i = 0
naip_names = []
if naip_year == "2016":
    # NAIP files have prefix
    naip_prefix = "ky_2ft_naip_2016_"
    extension = "jpg"
elif naip_year == "2018":
    naip_prefix = "KY_2FT_NAIP_2018_"
    extension = "tif"
elif naip_year == "2006":
    naip_prefix = "ky_2ft_naip_2006_"
    extension = "jpg"
for row in cursor:
    url = row[0].replace("_2016_", f"_{naip_year}_")
    if naip_year == "2006":
        name = row[1].lower()
    else:
        name = row[1]
    naip_names.append(name)
    print(f'Downloading {url}')
    with urllib.request.urlopen(url) as response: 
        with open(f'{downloads}{name}.zip', 'wb') as outFile:
            data = response.read()
            outFile.write(data) 
    with ZipFile(f'{downloads}{name}.zip', 'r') as zip: 
        zip.extractall(f'{downloads}{name}')
    print(f'{downloads}{name}\\{naip_prefix}{name}.{extension}')
    arcpy.CopyRaster_management (f'{downloads}{name}\\{naip_prefix}{name}.{extension}', name)
    i += 1
    
# If multiple NAIPs, then mosaic to new raster and clip
arcpy.Delete_management (f'temp')
if len(naip_names) > 1:
    arcpy.MosaicToNewRaster_management (naip_names, f'{root}\\{project}\\{geodb}', "temp", None, "8_BIT_UNSIGNED", None, 3)
    arcpy.Clip_management ('temp', '#', f'{project}_naip', 'domain')
else:
    arcpy.Clip_management (naip_names[0], '#', f'{project}_naip', 'domain')
arcpy.Delete_management (f'temp')

# Extract LAS points in buffer and colorize
arcpy.ExtractLas_3d (f'{lidar}trees', lidar, f'{project}_{buffer_distance}ft', "#", "#", "_extract", "#", "#", True, f'{lidar_extract}temp.lasd')
arcpy.ColorizeLas_3d (f'{lidar_extract}temp.lasd', f'{project}_naip', 'RED Band_1; GREEN Band_2; BLUE Band_3', lidar_color, "_color", "#",  "#",  "#",  "#", True, f'{lidar_color}{project}_rgb.lasd')

# Create DSM of cliffs over 30 feet in 30-ft diameter neighborhood from bare-earth DEM
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(f'{project}_dem_5ft', neighborhood, "RANGE")
outFocalStat.save("focal_stats_30ft")
cliffs_over_30ft = arcpy.sa.Con(outFocalStat > 30, outFocalStat)
cliffs_over_30ft.save("cliffs_over_30ft")

# Render point cloud with Potree
for i in las_names:
    with open(f'{root}\\{project}\\potree_las_list.txt', 'a+') as outFile:
        a = i.replace("\\lidar\\", "\\lidar_color\\")
        b = a.replace(".las", "_extract_color.las")
        print(b)
        outFile.write(f"{b}\n")
print(f"Using: {potree_tools} --list-of-files {root}\\{project}\\potree_las_list.txt -o {root}\\{project}\\potree -p index")
completed = subprocess.run(f"{potree_tools} --list-of-files {root}\\{project}\\potree_las_list.txt -o {root}\\{project}\\potree -p index")


c:\MontgomeryGIS\SkyBridge2\downloads\N053E177\KY_2FT_NAIP_2018_N053E177.tif
c:\MontgomeryGIS\SkyBridge2\downloads\N054E177\KY_2FT_NAIP_2018_N054E177.tif


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000157: Input and target dataset should have the same number of bands
Failed to execute (MosaicToNewRaster).
